We first initialize the system and set parameters.

In [1]:
import numpy as np
from functions import kalman_filter, rts_smoother, compute_expectations

# Dimensions
N = 2  # state: [temp, humidity]
M = 2  # observed: [temp, humidity]
T = 100  # timesteps

# State transition (slow drift)
F = np.array([
    [1.0, 0.0],
    [0.0, 0.95]
])

# Observation model (we directly observe both)
H = np.eye(2)

# Process noise (weather randomness)
Q = np.array([
    [0.1, 0.0],
    [0.0, 0.05]
])

# Measurement noise (sensor noise)
R = np.array([
    [0.5, 0.0],
    [0.0, 0.3]
])


Construct the actual model by letting the sytem evolve and by adding noise.

In [2]:
x_true = np.zeros((T, N))
y_obs = np.zeros((T, M))

# Initial true state
x_true[0] = np.array([20.0, 0.6])  # 20°C, 60% humidity

for t in range(1, T):
    x_true[t] = F @ x_true[t-1] + np.random.multivariate_normal(np.zeros(N), Q)

for t in range(T):
    y_obs[t] = H @ x_true[t] + np.random.multivariate_normal(np.zeros(M), R)


Fit using Kalman filter

In [ ]:
x0 = np.array([18.0, 0.5])     # initial guess
P0 = np.eye(N) * 5.0          # very uncertain initially

x_filt, P_filt, x_pred, P_pred = kalman_filter(
    y=y_obs,
    F=F,
    Q=Q,
    H=H,
    R=R,
    x0=x0,
    P0=P0
)

x_smoothed, P_smoothed, C = rts_smoother(
    x_filt, P_filt, x_pred, P_pred, F
)

Ex, Exx, Exx_tm1 = compute_expectations(x_smoothed, P_smoothed, C)

Plot estimated data vs. real data

In [4]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2)
"""
(1,1) will plot first component of state
(1,2) second component
"""

fig.add_trace(go.Scatter(
    x=list(range(T)),
    y=x_filt[:, 0],
    name="Temperature (state)"
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=list(range(T)),
    y=x_filt[:, 1],
    name="Humidity (state)"
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=list(range(T)),
    y=x_true[:, 0],
    name="Temperature (state) (real)"
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=list(range(T)),
    y=x_true[:, 1],
    name="Humidity (state) (real)"
), row=1, col=2)

fig.show()



Now we do the same thing, but with the RTS smoother added. We should see a better approximation of the real state.